# `jetto-mobo`: simple Bayesian optimisation for designing plasma profiles

Welcome to this tutorial on using Bayesian optimisation for plasma profile design! 
Our package, `jetto-mobo`, is designed to make using state-of-the-art model-based optimisation routines as painless as possible.

Documentation: https://jetto-mobo.readthedocs.io/en/latest/

Pre-print: https://arxiv.org/abs/2310.02669

<div>
<center>
<img src="assets/diagram_1.png" width="50%">
</center>
</div>

## Contents

1. Overview of the method

2. Running JETTO (or not)

3. Parameterising input profiles (+ exercise)

4. Formulating objective functions (+ exercise)

Exercise: Interim tests

5. Generating initial samples

6. Fitting a model (+ exercise)

7. Optimising an acquisition function (+ exercise)

8. Extracting the Pareto optimal solutions

Exercise: Bringing it all together

In [ ]:
# This may take a minute or so to run
import torch

## 1. Overview of the method

Bayesian optimisation is a *model-based* optimisation scheme, that uses a probabilistic model to make decisions about what points to trial next. It has been shown to outperform other gradient-free optimisation schemes in a wide variety of tasks. Check out the [BayesOpt section of the documentation](https://jetto-mobo.readthedocs.io/en/latest/bayesopt.html) for more detail!

In short, we use a Gaussian process (GP) to learn the mapping from input parameters (e.g. the shape of an ECRH profile) to objective functions (e.g. the shape of the $q$-profile). We can then use the GP's predictive distribution to optimise an acquisition function, which tells us which points we should try next.


<div>
<center>
<img src="assets/bo_flowchart.svg" width="75%">
</center>
</div>

Our package is specifically designed for *multi-objective* optimisation, where there are multiple competing objectives that need to be optimised. Tackling multi-objective optimisation tasks in the right way can produce really illuminating results that can help shape the decision-making process, as we can gain an understanding about how the objectives interact.
Again, there's a bit more detail in the [multi-objective optimisation section of the documentation](https://jetto-mobo.readthedocs.io/en/latest/multiobjective.html).


## 2. Running JETTO (or not)

Because JETTO takes ~3 hours to converge to a new steady-state configuration, we've provided a quick-and-dirty replacement that makes up a semi-plausible $q$ profile from an input ECRH profile. In this notebook we'll use this replacement, `workshop_tools.run_toy`, instead of the 'real' function, `jetto_mobo.simulation.run`, which launches JETTO in a Singularity container. 

Often you want to launch many JETTO runs in parallel, so we also provide a toy version of `run_many`.

The signatures and behaviour of the functions are exactly the same (although the toy ones are less reliably realistic!), so when you want to use the real function you can do a direct swap.


In [ ]:
 # If you were running the full JETTO, you'd use `from jetto_mobo.simulation import run`
from workshop_tools import run_toy
 # If you were running the full JETTO, you'd use `from jetto_mobo.simulation import run_many`
from workshop_tools import run_many_toy

`run` and `run_toy` take a `RunConfig` object, an output directory, and a JETTO Singularity image, and return a `JettoResults` object:

In [ ]:
help(run_toy)

`run_many` and `run_many_toy` take a JETTO singularity image, and a `dict` of `RunConfig`s and corresponding output directories, returning a list of `JettoResults`:

In [ ]:
help(run_many_toy)

You might already have a way of generating JETTO `RunConfig` objects. We found that it was a bit finicky, so we wrote a helper function to do it for me. This copies the template to a directory, and creates a `RunConfig` for the new directory:

In [ ]:
from workshop_tools import create_config_toy # Again, if you were using the full JETTO, you'd use `from jetto_mobo.simulation import create_config`

test_config = create_config_toy(template="assets/spr45-qlknn", directory="runs/test")

The `RunConfig` returned by `create_config` also has an extra property, `exfile`, that stores the path to the binary exfile. Modifying the input profiles is done by loading, editing and saving the exfile:

In [ ]:
import jetto_tools.binary
import numpy as np

# Load the exfile
exfile = jetto_tools.binary.read_binary_file(test_config.exfile)

# Make an EC profile that's a Gaussian bump centred on the origin
# Note that we modify the 0th time slice of the EC profile
exfile["QECE"][0] = np.exp(-(exfile["XRHO"][0]**2)/0.01)

# Save the exfile
jetto_tools.binary.write_binary_exfile(exfile, test_config.exfile)

The `jetto_mobo` package uses `asyncio` when running the JETTO containers - this is way of managing non-blocking multiprocess execution.

In a 'normal' Python script, you would need to run these functions using `asyncio.run`, as explained in the docstrings. However, in an IPython environment (such as this Jupyter notebook), you instead run the functions using the `await` keyword:

In [ ]:
from workshop_tools import plot_profiles # Helper function for plotting QECE/q profiles

results = await run_toy(
        run_config=test_config,
        run_directory="runs/test",
        jetto_image="", # Leave this as a blank string - in practice, you would specify the path to the JETTO Singularity image
    )

profiles = results.load_profiles()
plot_profiles(profiles)

Hooray! We've successfully run the toy version of JETTO to simulate a $q$-profile from an ECRH profile.

The `JettoResults` object returned by `run_toy` has `profiles.CDF` and `timetraces.CDF` with the following entries set:

```
profiles["QECE"][-1] # Set to the ECRH profile provided as an input (i.e. profiles["QECE"][0])
profiles["Q"][-1] # Set to the toy model's prediction of the q-profile
timetraces["Q0"][-1] # Set to the on-axis value of the model's prediction
timetraces["QMIN"][-1] # Set to the minimum of the model's prediction
timetraces["ROQM"][-1] # Set to the normalised radial location of the model's prediction
```

Any other entries will not have been modified from the JETTO run template.

## 3. Parameterising input profiles

`jetto-mobo` is designed for optimisation of plasma profiles. It provides a Python decorator to ensure that functions representing a profile are of the correct form. If a function does not match the decorator signature *exactly* (including type hints!), it will throw an error.

In [ ]:
from jetto_mobo.inputs import plasma_profile

help(plasma_profile)

Example of incorrect use:

In [ ]:
@plasma_profile
def gaussian_profile(x, p):
    variance = p[0]
    mean = p[1]
    scale = p[2]
    return scale * np.exp(-(x - mean)**2 / variance)

Example of correct use:

In [ ]:
@plasma_profile
def gaussian_profile(xrho: np.ndarray, parameters: np.ndarray) -> np.ndarray:
    """
    **Remember to write an informative docstring!**
    
    A Gaussian profile with specified variance, mean and scale.
    
    Parameters
    ----------
    xrho : np.ndarray
        Normalised radial coordinate.
    parameters : np.ndarray
        Array of parameters, in the order [variance, mean, scale].
        
    Returns
    -------
    np.ndarray
        Gaussian profile.
    """
    variance = parameters[0]
    mean = parameters[1]
    scale = parameters[2]
    return scale * np.exp(-(xrho - mean)**2 / variance)

#### Things to think about 

When you're designing an optimisation task, the parameterisation of the input plays an important role in shaping the solutions that you generate. Some things you need to think about are:

1. How versatile is it?

    Does the parameterisation permit a wide range of input profiles to be represented? We want to ensure that the (unknown) optimal profile can be represented by our parameterisation, without pre-determining the kind of solution we expect.

    At the same time, are most of the profiles vaguely sensible? We don't want to waste time exploring regions of space that are *a priori* unlikely to perform well.


2. How unique is the mapping?

    Does each input profile correspond to exactly one set of parameters? If not, then we end up duplicating regions of search space, which reduces efficiency.


3. How continuous is the mapping from parameters to objectives?

    We want regions that are close in parameter space to be close in objective space, because this makes it much easier for the model to capture the mapping accurately. If you tweak the parameters a little bit, the resulting objective values should also move a little bit.

#### Exercise: ECRH parameterisation

In the cell below, define a parameterisation suitable for representing ECRH profiles. Your profile should be constrained to $[0, 1]$.
Remember to decorate it with `@plasma_profile`!

Also define a $2 \times M$ array, where $M$ is the number of parameters, representing the lower and upper bounds of the allowed parameter ranges.

Use `plot_ecrh_profile` to visualise your parameterisation.

In [ ]:
import numpy as np
from jetto_mobo.inputs import plasma_profile
from workshop_tools import plot_ecrh_profile
bounds = np.array([[1e-4, 0, 0.1], [1e-2, 1, 1]])
plot_ecrh_profile(gaussian_profile, bounds)

# Define your ECRH profile
# ...

# Define the parameter bounds
# parameter_bounds = ...

# Visualise the profile
# ...

## 4. Formulating objective functions

An objective function takes the output of a JETTO run and returns a vector corresponding to the performance of the run against various different metrics.
`jetto-mobo` provides another decorator to enforce compatibility:

In [ ]:
from jetto_mobo.objectives import objective

help(objective)

The decorator is designed for vector objectives, but also supports scalar ones.
A vector usage example:

In [ ]:
import netCDF4 as nc
import numpy as np
from jetto_mobo.objectives import objective 
from jetto_tools.results import JettoResults

def q_is_positive(profiles: nc.Dataset):
    # profiles["Q"] is a 2D array of shape (time, radius)
    # profiles["Q"][-1] is the last time slice
    return 1 if np.all(profiles["Q"][-1].data) > 0 else 0

def q0_is_greater_than_2(timetraces: nc.Dataset):
    # Some variables are stored in the timetraces Dataset
    return 1 if timetraces["Q0"][-1].data > 2 else 0

@objective
def my_fancy_objective(results: JettoResults) -> np.ndarray:
    try:
        # Sometimes this throws an error, when something has gone wrong with JETTO
        # Putting it in a try/except block means that we can handle these cases
        profiles = results.load_profiles()
        timetraces = results.load_timetraces()
    except:
        # There are a few options for what to do with JETTO failures
        # In this case, we choose to treat them as badly-scoring points,
        # rather than discarding them entirely
        return np.array([0, 0])
    
    return np.array(
        [
            q_is_positive(profiles),
            q0_is_greater_than_2(timetraces),
        ]
    )

#### Things to think about

Both the choice of objective criteria and their mathematical formulation have a strong impact on the kinds of solutions that are produced. Some things to consider when designing objective functions are:

1. Are any of these objectives measuring the same thing?

    As the dimensionality of the objective space increases, everything becomes much more difficult. Keeping the dimensionality as low as possible ensures that the algorithm explores sufficiently.


2. How do I want the objective value to decay?

    The GP surrogate model of the input-objective mapping works best if the objective values vary smoothly.
    It's also useful to have some objective value everywhere in space, rather than it being 0 in a lot of regions.

3. Are my objectives directly comparable?

    This is important for two reasons. Firstly, the acquisition function that our package uses is currently qNEHVI, which involves computing hypervolumes in objective space. If the objectives vary on dramatically different scales, the increase in hypervolume size for a modest improvement in one objective may outweigh the improvement achievable in another objective.
    
    Secondly, when a human comes to interpret the solutions, it is much easier to understand if all the objectives vary similarly.
    
    
4. Could any of these objectives be reformulated as constraints?

    In general, constraints are easier to model than objectives, thanks to the curse of dimensionality.
    We don't cover constrained BO here, but it's quite easy to extend to (e.g. see the [BoTorch docs](https://botorch.org/tutorials/constrained_multi_objective_bo)).

#### Exercise: $q$-profile objectives

In this workshop, we're tackling the ECRH-$q$ optimisation problem, and so all our metrics will relate to the shape of the $q$ profile. Our toy model only supports computing $q$ metrics that use `QECE` and `Q` from JETTO profiles file, and `Q0`, `QMIN`, and `ROQM` from the timetraces file.

Ideally, future work would also explore other impacts of the ECRH profile, but for the time being we'll stick with looking at $q$.

In the cell below, define a vector objective function called `q_vector_objective` with at least 3 components that relate to properties of the $q$-profile. You might want to define a common transform that is used to rescale and normalise all the objectives, so that they vary in the same way.

In [ ]:
import numpy as np
from jetto_mobo.objectives import objective 
from jetto_tools.results import JettoResults

# Define your objective function
# ...

## Exercise: interim tests

Modify the below code to test your input and objective functions!

In [ ]:
from workshop_tools import run_many_toy, create_config_toy, plot_results
import numpy as np
from typing import Iterable
from jetto_tools.results import JettoResults

# Define a function that:
# 1. Takes a set of parameters (of shape (batch_size, n_parameters))
# 2. Creates a set of batch_size RunConfigs using create_config_toy
#    (ie uses create_config_toy, and then modifies the ECRH profile in
#     each RunConfigs' exfile using the parameters and your ECRH function)
# 3. Runs the simulations using run_many_toy, to get a list of batch_size JettoResults objects
# 4. Computes the objective function values using the results
# 5. Returns a tuple containing:
#      - The ECRH profiles as a np.ndarray of shape (batch_size, any)
#      - The q profiles as a np.ndarray of shape (batch_size, any)
#      - The objective function values as a np.ndarray of shape (batch_size, n_objectives)
def evaluate(parameters: np.ndarray) -> (np.ndarray, np.ndarray, np.ndarray):
    # Convert the parameters to a list of ECRH profiles
    # ecrh = ...
    # Convert the ECRH profiles to a list of RunConfigs
    # ...
    # Run the simulations
    # ...
    # Extract the q profiles
    # q = ...
    # Compute the objective function values
    # objective_values = ...
    return ecrh, q, objective_values

# Define the parameter bounds
# parameter_bounds = ...

# Generate some random parameters
batch_size = 3
rng = np.random.default_rng(42)
parameters = rng.uniform(parameter_bounds[:, 0], parameter_bounds[:, 1], size=(batch_size, parameter_bounds.shape(1)))

# Evaluate
ecrh, q, objective_values = evaluate(parameters)

# Plot the results, adding the objective labels
objective_labels = [...]
plot_results(ecrh, q, objective_values, objective_labels)

## 5. Fitting a model

BayesOpt uses a probabilistic model (a Gaussian process, or GP) in selecting the next candidates to trial.
The GP provides a distribution over the predicted performance (objective values) of a candidate (in our case, ECRH profile).

In [ ]:
from jetto_mobo.surrogate import fit_surrogate_model

help(fit_surrogate_model)

Fitting a Gaussian process can be accelerated using a GPU; we use PyTorch to allow cross-compatibility between GPUs and CPUs. The downside of this is that it requires a bit more fiddling, and requires the use of the `torch` library.

Quick demo of using `torch.tensor`:

In [ ]:
import numpy as np
import torch

a = np.array([1.1, 2.2, 3.3])

# Convert an array to a PyTorch tensor on the CPU
a_cpu_tensor = torch.tensor(a, dtype=torch.float32, device=torch.device("cpu"))

# Convert an array to a PyTorch tensor on an NVIDIA GPU
# a_gpu_tensor = torch.tensor(a, dtype=torch.float32, device=torch.device("cuda:0"))

# Convert a tensor on the CPU to a tensor on an NVIDIA GPU
# a_gpu_tensor = a_cpu_tensor.to(device=torch.device("cuda:0"))

# Convert a tensor back to a numpy array
a_again = a_cpu_tensor.detach().cpu().numpy()

In our case, `X` will be the ECRH parameterisation parameters, `X_bounds` the ECRH parameter bounds, and `Y` the objective values.

Let's demonstrate the fitting of a GP.

In [ ]:
from jetto_mobo.surrogate import fit_surrogate_model
import torch
import numpy as np
import plotly.graph_objects as go 
import plotly.io as pio
pio.renderers.default='notebook'

def himmelblau(x1, x2):
    x1 = 8 * x1 - 4
    x2 = 8 * x2 - 4
    return np.clip(300 - (x1**2 + x2 - 11)**2 - (x1 + x2**2 - 7)**2, 0, None)

# Generate some 2D data
input_bounds = np.array([[0, 0], [1, 1]])
x1, x2 = np.meshgrid(np.linspace(input_bounds[0, 0], input_bounds[1, 0], 100), np.linspace(input_bounds[0, 1], input_bounds[1,1], 100))
x = np.stack([x1.flatten(), x2.flatten()], axis=-1)
y = himmelblau(x[:, 0], x[:, 1])

ground_truth_figure = go.Figure(
    go.Contour(
        x=x[:, 0],
        y=x[:, 1],
        z=y,
        contours_coloring="heatmap",
    ),
    layout_title="Example function"
)
ground_truth_figure.update_layout(
    xaxis_title="x1",
    yaxis_title="x2",
)
ground_truth_figure.show()

In [ ]:
# Subsample the data
rng = np.random.default_rng(42)
sample_indices = rng.choice(np.arange(x.shape[0]), size=50, replace=False)
x_sample = x[sample_indices, :]
y_sample = y[sample_indices]

ground_truth_figure.add_trace(
    go.Scatter(
        x=x_sample[:, 0],
        y=x_sample[:, 1],
        mode="markers",
        showlegend=False,
        marker_color="lightgreen"
    )
)
ground_truth_figure.update_layout(
    xaxis_range=[0, 1],
    yaxis_range=[0, 1],
)
ground_truth_figure.show()

In [ ]:
# Fit a surrogate model to the samples
model = fit_surrogate_model(
    X=torch.tensor(x_sample),
    X_bounds=torch.tensor(input_bounds),
    Y=torch.tensor(y_sample).unsqueeze(-1),
)

# Because this is a 2D function, we can visualise the surrogate quite easily
# We plot the mean of the surrogate model's predictions, without showing the uncertainty
model_output = model(torch.tensor(x)).mean.detach().cpu().numpy()

go.Figure(
    go.Contour(
        x=x[:, 0],
        y=x[:, 1],
        z=model_output,
        contours_coloring="heatmap",
    ),
    layout_title="Surrogate's prediction"
).show()
    

Here, we had a true function (the 2D Himmelblau function, $f: \mathbb{R}^2 \to \mathbb{R}$), made a few observations (green), and fit the GP model to the observations. There's no trickery here, just statistics - fitting the GP is done just by inverting and multiplying some matrices!

In BayesOpt, what we have instead is a true function (the mapping from $p$ input EC parameters to $n$ objective values $\phi: \mathbb{R}^p \to \mathbb{R}^n$), which we can *selectively* observe. In the next sections we'll talk about how the observation selection process.

## 6. Generating initial candidates

The GP model requires some initial data to get started. Ideally this data will be widely spread throughout the input space. One method for generating initial samples is Sobol sampling, a quasirandom method that generates more evenly distributed samples than other methods (such as a pure random sampling). Other methods exist, such as Latin Hypercube sampling; for some reason, everyone seems to use Sobol in BayesOpt.

| Random sampling | Sobol sampling |
| :-------------: | :------------: |
| ![Random sampling](assets/random.svg) | ![Sobol sampling Image](assets/sobol.svg)|

`jetto-mobo` provides a wrapper for a Sobol sampling function:

In [ ]:
from jetto_mobo.acquisition import generate_initial_candidates

help(generate_initial_candidates)

## Exercise: initialising a GP model

In this exercise, you need to generate some initial candidates using Sobol sampling, and then use the candidates to initialise a GP surrogate model of the objective function.

In [ ]:
from jetto_mobo.acquisition import generate_initial_candidates
from workshop_tools import plot_ecrh_profile, plot_results, run_many_toy

# Generate some initial candidates using your parameter_bounds vector
# and the generate_initial_candidates function
# Remember to convert the bounds to a torch tensor!
# parameters = ...

# Run the candidates through the simulation using your evaluate function
ecrh, q, objective_values = evaluate(parameters)

# Visualise the results using plot_results
plot_results(ecrh, q, objective_values, objective_labels)

We'll use these initial samples as our starting point for optimisation. To do so, we first need to fit the GP model to them.

In [ ]:
from jetto_mobo.surrogate import fit_surrogate_model

# Fit a surrogate model to the results
# The surrogate model is learning the mapping from ECRH profile parameters 
# to objective values
# ecrh_q_model = ...

## 7. Optimising an acquisition function

The key step in BayesOpt is using the GP model to inform the selection of the next points. As a Gaussian process provides estimates of the uncertainty in its predictions of the performance of future points, an *acquisition function* can decide whether to try points that are known to be good, or points where the performance is unknown.

In [ ]:
from jetto_mobo.acquisition import generate_trial_candidates

help(generate_trial_candidates)

Although there are a lot of options here, you really don't need to worry about most of them. All of the optional arguments have sensible defaults.

Because we're looking at multi-objective optimisation, we use `jetto_mobo.acquisition.qNoisyExpectedHypervolumeImprovement` as our acquisition function.

### Exercise: optimising an acquisition function

Using the `ecrh_q_model` that you defined previously, generate, evaluate and visualise a new set of trial candidates by optimising the qNEHVI acquisition function.

In [ ]:
from jetto_mobo.acquisition import generate_trial_candidates, qNoisyExpectedHypervolumeImprovement
from workshop_tools import plot_results, run_many_toy

# Generate some trial candidates by optimising the qNEHVI acquisition function
# using your parameter_bounds vector and the generate_trial_candidates function
# Remember to convert the bounds to a torch tensor!
# new_parameters = ...

# Run the candidates through the simulation using your evaluate function
new_ecrh, new_q, new_objective_values = evaluate(new_parameters)

# Visualise the results using plot_results
plot_results(new_ecrh, new_q, new_objective_values, objective_labels)

## 8. Extracting the Pareto optimal solutions

In a multi-objective problem, there are no single best solutions. Instead, solutions lie on a surface - called the Pareto front - that represents the optimal tradeoffs between objectives. These solutions are called Pareto optimal.

qNEHVI and other multi-objective acquisition functions seek to find solutions that populate the Pareto front. During optimisation, the algorithm should hopefully find better and better solutions, pushing the Pareto front outwards.

Once optimisation is complete, we need to filter all of the solutions that have been observed, to extract the set of solutions that are Pareto optimal.

In [ ]:
from jetto_mobo.utils import get_pareto_dominant_mask
help(get_pareto_dominant_mask)

In [ ]:
# Concatenate the old and new results
all_parameters = np.concatenate([parameters, new_parameters], axis=0)
all_ecrh = np.concatenate([ecrh, new_ecrh], axis=0)
all_q = np.concatenate([q, new_q], axis=0)
all_objective_values = np.concatenate([objective_values, new_objective_values], axis=0)

# Get the Pareto-dominant points
is_pareto_dominant = get_pareto_dominant_mask(all_objective_values)

# Visualise the results using plot_results
plot_results(all_ecrh[is_pareto_dominant], all_q[is_pareto_dominant], all_objective_values[is_pareto_dominant], objective_labels)

## Exercise: Bringing it all together

Congratulations! You're now ready to write a full Bayesian optimisation routine.

Remember, the method is:

1. Define a parameterisation for your input
2. Define your objective values
3. Generate some initial Sobol samples of the input
4. Observe the objective values of the initial samples
5. Fit a GP to the initial data
6. Optimise the acquisition function to produce some new candidate parameters
7. Observe the objective values of the new candidates
8. Fit a GP to all the data seen to date
9. Repeat from (6) until a good enough solution (or your compute budget) is reached!
10. Filter out the Pareto optimal results

In the cell below, write a loop to run steps 6-9 for 10 steps of 10 candidates, storing the results as you go. Then, filter out the Pareto optimal results and display them with `plot_results`.
